In [1]:
from langchain_core.documents import Document
from ragbase.model import create_reranker, create_llm
from langchain.retrievers.document_compressors.chain_filter import LLMChainFilter
from langchain.globals import set_verbose

set_verbose(True)

In [2]:
reranker = create_reranker()

In [3]:
documents = [
    Document("""Not since Porsche blew air cooling out the back door in 1999 with the 996-
generation 911 has there been a bigger change that will upset more Porsche fans
than the hybridization of the 911. Fans, however, are not necessarily owners, and
those with the means will not mind the 21st-century tech one bit, because if
Porsche didn't tell anyone this updated 992.2 was a hybrid, no one would know by
how it drives.
"""),
    Document("""For now, the new 3.6-liter flat-six operating in perfect stoichiometry all the time,
its electrified turbocharger, and eight-speed dual-clutch gearbox with an integral
electric motor are limited to the GTS, which used to be powered by a jazzed-up
version of the twin-turbo 3.0-liter in the base Carrera.
"""),
    Document("""
Anyway, the screen is cheaper, but at least Porsche does some cool stuff with it.
There are seven different views, including a classic five-gauge cluster, but the most
interesting of them is a track-focused mode that clocks the tach so that the redline
is near 12 o'clock. Porsche would have scored more points if it had kept the 
orientation of the tach numbers so that they locked with the twist, like a real
clocked tach out of a 917.
"""),
    Document("""The bad news is that the GTS coupe starts at $166,895. The back seat is no longer
standard, though adding the +2 second row is a no-cost option. That's mega
money, but until Porsche sees sales drop, you better believe it will keep raising its
prices. That's just basic supply-demand, and plenty of people will line up for this
water-cooled hybrid.
""")
]

## Reranker

In [4]:
reranked_documents = reranker.compress_documents(documents, "What is the price?")

In [5]:
first_result = reranked_documents[0]
print(first_result.__dict__.keys())
print("score:", first_result.metadata["relevance_score"])
print(first_result.page_content)

dict_keys(['id', 'metadata', 'page_content', 'type'])
score: 0.12178549
The bad news is that the GTS coupe starts at $166,895. The back seat is no longer
standard, though adding the +2 second row is a no-cost option. That's mega
money, but until Porsche sees sales drop, you better believe it will keep raising its
prices. That's just basic supply-demand, and plenty of people will line up for this
water-cooled hybrid.



## LLM Chain Filter

In [6]:
chain_filter = LLMChainFilter.from_llm(create_llm())
filtered_documents = chain_filter.compress_documents(documents, "How many seats in the car?")



> Entering new LLMChain chain...

> Entering new LLMChain chain...
Prompt after formatting:
Given the following question and context, return YES if the context is relevant to the question and NO if it isn't.

> Question: How many seats in the car?
> Context:
>>>
For now, the new 3.6-liter flat-six operating in perfect stoichiometry all the time,
its electrified turbocharger, and eight-speed dual-clutch gearbox with an integral
electric motor are limited to the GTS, which used to be powered by a jazzed-up
version of the twin-turbo 3.0-liter in the base Carrera.

>>>
> Relevant (YES / NO):

Prompt after formatting:
Given the following question and context, return YES if the context is relevant to the question and NO if it isn't.

> Question: How many seats in the car?
> Context:
>>>
Not since Porsche blew air cooling out the back door in 1999 with the 996-
generation 911 has there been a bigger change that will upset more Porsche fans
than the hybridization of the 911. Fans, however, a

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 


> Finished chain.

> Finished chain.

> Finished chain.


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



> Finished chain.


In [7]:
print(len(filtered_documents))
print(filtered_documents[0].page_content)

1
The bad news is that the GTS coupe starts at $166,895. The back seat is no longer
standard, though adding the +2 second row is a no-cost option. That's mega
money, but until Porsche sees sales drop, you better believe it will keep raising its
prices. That's just basic supply-demand, and plenty of people will line up for this
water-cooled hybrid.

